# 1. Dataset preprocessing
### 1-1. Data structure
         ─/data/freetalk_senior
            ├─2.Validation
            │  ├─1.labeled_data
            │  │  ├─1.AI챗봇
            │  │  │  └─1.AI챗봇_라벨링_자유대화(노인남여)_VALIDATION.zip | 50 MB | 48677
            │  │  ├─2.음성수집도구
            │  │  │  └─2.음성수집도구_라벨링_자유대화(노인남여)_VALIDATION.zip | 105 MB | 48678
            │  │  ├─3.스튜디오
            │  │  │  └─3.스튜디오_라벨링_자유대화(노인남여)_VALIDATION.zip | 20 MB | 48679
            │  │  └─4.AI스피커
            │  │      └─4.AI스피커_라벨링_자유대화(노인남여)_VALIDATION.zip | 13 MB | 48680
            │  └─2.raw_data
            │      ├─1.AI챗봇
            │      │  └─1.AI챗봇_자유대화(노인남여)_VALIDATION.zip | 7 GB | 48681
            │      ├─2.음성수집도구
            │      │  └─2.음성수집도구_자유대화(노인남여)_VALIDATION.zip | 21 GB | 48682
            │      ├─3.스튜디오
            │      │  └─3.스튜디오_자유대화(노인남여)_VALIDATION.zip | 3 GB | 48683
            │      └─4.AI스피커
            │          └─4.AI스피커_자유대화(노인남여)_VALIDATION.zip | 2 GB | 48684
            └─1.Training
                ├─1.labeled data
                │  ├─1.AI챗봇
                │  │  └─1.AI챗봇_라벨링_자유대화(노인남여)_TRAINING.zip | 405 MB | 48663
                │  ├─2.음성수집도구
                │  │  └─2.음성수집도구_라벨링_자유대화(노인남여)_TRAINING.zip | 718 MB | 48664
                │  ├─3.스튜디오
                │  │  └─3.스튜디오_라벨링_자유대화(노인남여)_TRAINING.zip | 156 MB | 48665
                │  └─4.AI스피커
                │      └─4.AI스피커_라벨링_자유대화(노인남여)_TRAINING.zip | 92 MB | 48666
                └─2.raw_data
                   ├─1.AI챗봇
                   │  ├─1.AI챗봇_1_자유대화(노인남여)_TRAINING.zip | 28 GB | 48667
                   │  └─1.AI챗봇_2_자유대화(노인남여)_TRAINING.zip | 29 GB | 48668
                   ├─2.음성수집도구
                   │  ├─2.음성수집도구_1_자유대화(노인남여)_TRAINING.zip | 29 GB | 48669
                   │  ├─2.음성수집도구_2_자유대화(노인남여)_TRAINING.zip | 28 GB | 48670
                   │  ├─2.음성수집도구_3_자유대화(노인남여)_TRAINING.zip | 27 GB | 48671
                   │  ├─2.음성수집도구_4_자유대화(노인남여)_TRAINING.zip | 29 GB | 48672
                   │  ├─2.음성수집도구_5_자유대화(노인남여)_TRAINING.zip | 28 GB | 48673
                   │  └─2.음성수집도구_6_자유대화(노인남여)_TRAINING.zip | 27 GB | 48674
                   ├─3.스튜디오
                   │  └─3.스튜디오_자유대화(노인남여)_TRAINING.zip | 27 GB | 48675
                   └─4.AI스피커
                      └─4.AI스피커_자유대화(노인남여)_TRAINING.zip | 20 GB | 48676

### 1-2. Data가 너무 크니 일부만 test 해본다


In [ ]:
# import data - samples
from datasets import load_dataset

data_files= {
    "train_input": r"/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_1_자유대화(노인남여)_TRAINING/노인남여_노인대화07_F_1527804469_64_강원_실내/*",
    "train_label": r"/data/freetalk_senior/1.Training/labeled_data/1.AI챗봇/1.AI챗봇_라벨링_자유대화(노인남여)_TRAINING/노인남여_노인대화07_F_1527804469_64_강원_실내/*.json",
}

# json file 구조가 "data": [{data1}, {data2}, ...] 처럼 되어있지 않고 
# {data1}, {data2}, ... 이므로 field="data" 와 같이 필드 지정하면 오류남
dataset_input = load_dataset("audiofolder", data_files=data_files["train_input"])
dataset_label = load_dataset("json", data_files=data_files["train_label"])

print(dataset_input, dataset_label)

In [ ]:
# def: np.c_(columns)
from datasets import Dataset, DatasetDict

def column_wise_concat(d1, d2):
    assert len(d1) == len(d2),  "Datasets must have the same len"
    d1_columns = {k: d1["train"][k] for k in d1["train"].features}
    d2_columns = {k: d2["train"][k] for k in d2["train"].features}

    combined_columns = {**d1_columns, **d2_columns}

    res = Dataset.from_dict(combined_columns)
    return DatasetDict({"train": res})

senior_dataset = column_wise_concat(dataset_input, dataset_label)
senior_dataset

# 모르겠다 label은 그냥 싹다 unzip 하자

In [ ]:
import os, glob
from datasets import load_dataset
from tqdm import tqdm

def unzip_json_label(file_dir='/data/freetalk_senior/'):
    labeled_data_files_TR = glob.glob(os.path.join(file_dir, "1.Training/labeled_data/**/*.zip"), recursive=True)
    labeled_data_files_VAL = glob.glob(os.path.join(file_dir, "2.Validation/labeled_data/**/*.zip"), recursive=True)

    extract_to_dir1 = []
    extract_to_dir2 = []
    for i, zip_data in enumerate(tqdm(labeled_data_files_TR)):
        extract_to_dir1.append(zip_data.replace('.zip', ''))
        os.system(f"mkdir '{extract_to_dir1[i]}'")
        os.system(f"unzip '{zip_data}' -d '{extract_to_dir1[i]}' > /dev/null 2>&1")
        # print(f"mkdir '{extract_to_dir[i]}'")
        # print(f"unzip '{zip_data}' -d '{extract_to_dir[i]}'")

    for i, zip_data in enumerate(tqdm(labeled_data_files_VAL)):
        extract_to_dir2.append(zip_data.replace('.zip', ''))
        os.system(f"mkdir '{extract_to_dir2[i]}'")
        os.system(f"unzip '{zip_data}' -d '{extract_to_dir2[i]}' > /dev/null 2>&1")
        # print(f"mkdir '{extract_to_dir[i]}'")
        # print(f"unzip '{zip_data}' -d '{extract_to_dir[i]}'")
         
    return   

In [ ]:
# UNIZP 완료 !!!
# unzip_json_label()

# 2. Load whole dataset

In [1]:
# recursive 한 zip file load 되는가 실험 -> 된다

In [1]:
import os
import glob
from datasets import load_dataset

# File path
training_file_dir = "/data/freetalk_senior/1.Training/"

training_raw_data = sorted(glob.glob(os.path.join(training_file_dir, "raw_data/**/*.zip"), recursive=True))
training_raw_data = training_raw_data[2:] # 앞의 2개 뺌

training_labeled_data = []
# training_labeled_data.append(sorted(glob.glob(os.path.join(training_file_dir, "labeled_data/1.*/**/*.json"), recursive=True)))
training_labeled_data.append(sorted(glob.glob(os.path.join(training_file_dir, "labeled_data/2.*/**/*.json"), recursive=True)))
training_labeled_data.append(sorted(glob.glob(os.path.join(training_file_dir, "labeled_data/3.*/**/*.json"), recursive=True)))
training_labeled_data.append(sorted(glob.glob(os.path.join(training_file_dir, "labeled_data/4.*/**/*.json"), recursive=True)))

In [21]:
training_raw_data

['/data/freetalk_senior/1.Training/raw_data/2.음성수집도구/2.음성수집도구_1_자유대화(노인남여)_TRAINING.zip',
 '/data/freetalk_senior/1.Training/raw_data/2.음성수집도구/2.음성수집도구_2_자유대화(노인남여)_TRAINING.zip',
 '/data/freetalk_senior/1.Training/raw_data/2.음성수집도구/2.음성수집도구_3_자유대화(노인남여)_TRAINING.zip',
 '/data/freetalk_senior/1.Training/raw_data/2.음성수집도구/2.음성수집도구_4_자유대화(노인남여)_TRAINING.zip',
 '/data/freetalk_senior/1.Training/raw_data/2.음성수집도구/2.음성수집도구_5_자유대화(노인남여)_TRAINING.zip',
 '/data/freetalk_senior/1.Training/raw_data/2.음성수집도구/2.음성수집도구_6_자유대화(노인남여)_TRAINING.zip',
 '/data/freetalk_senior/1.Training/raw_data/3.스튜디오/3.스튜디오_자유대화(노인남여)_TRAINING.zip',
 '/data/freetalk_senior/1.Training/raw_data/4.AI스피커/4.AI스피커_자유대화(노인남여)_TRAINING.zip']

In [3]:
[len(training_labeled_data[i]) for i in range(len(training_labeled_data))]

[595637, 133387, 80318]

In [22]:
sum([len(training_labeled_data[i]) for i in range(len(training_labeled_data))])

809342

### val
val_file_dir = "/data/freetalk_senior/2.Validation"
val_raw_data = sorted(glob.glob(os.path.join(val_file_dir, "raw_data/**/*.zip"), recursive=True))

val_labeled_data = []
val_labeled_data.append(sorted(glob.glob(os.path.join(val_file_dir, "labeled_data/1.*/**/*.json"), recursive=True)))
val_labeled_data.append(sorted(glob.glob(os.path.join(val_file_dir, "labeled_data/2.*/**/*.json"), recursive=True)))
val_labeled_data.append(sorted(glob.glob(os.path.join(val_file_dir, "labeled_data/3.*/**/*.json"), recursive=True)))
val_labeled_data.append(sorted(glob.glob(os.path.join(val_file_dir, "labeled_data/4.*/**/*.json"), recursive=True)))

## data들

    ['/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_1_자유대화(노인남여)_TRAINING.zip', # label 보다 한 개 적음
     '/data/freetalk_senior/1.Training/raw_data/1.AI챗봇/1.AI챗봇_2_자유대화(노인남여)_TRAINING.zip', 
     '/data/freetalk_senior/1.Training/raw_data/2.음성수집도구/2.음성수집도구_1_자유대화(노인남여)_TRAINING.zip',  
     '/data/freetalk_senior/1.Training/raw_data/2.음성수집도구/2.음성수집도구_2_자유대화(노인남여)_TRAINING.zip',
     '/data/freetalk_senior/1.Training/raw_data/2.음성수집도구/2.음성수집도구_3_자유대화(노인남여)_TRAINING.zip',
     '/data/freetalk_senior/1.Training/raw_data/2.음성수집도구/2.음성수집도구_4_자유대화(노인남여)_TRAINING.zip',
     '/data/freetalk_senior/1.Training/raw_data/2.음성수집도구/2.음성수집도구_5_자유대화(노인남여)_TRAINING.zip',
     '/data/freetalk_senior/1.Training/raw_data/2.음성수집도구/2.음성수집도구_6_자유대화(노인남여)_TRAINING.zip',
     '/data/freetalk_senior/1.Training/raw_data/3.스튜디오/3.스튜디오_자유대화(노인남여)_TRAINING.zip',
     '/data/freetalk_senior/1.Training/raw_data/4.AI스피커/4.AI스피커_자유대화(노인남여)_TRAINING.zip']

In [5]:
data_files = {
    "train_input": training_raw_data,    
    "train_label": training_labeled_data,
}

### val
data_files = {
    "val_input": val_raw_data,    
    "val_label": val_labeled_data,
}

In [6]:
# Load Dataset
from datasets import Features, Audio

# load_dataset 이 새로 만들어 버리는 label column 때문에 에러 발생하여 features명시
features = Features({
    'audio': Audio()
})

train_input = load_dataset("audiofolder", data_files={"train": data_files["train_input"]}, 
                                            features=features)
train_input

DatasetDict({
    train: Dataset({
        features: ['audio'],
        num_rows: 809342
    })
})

### val
val_input = load_dataset("audiofolder", data_files={"validation": data_files["val_input"]}, 
                                            features=features)
val_input.shape

In [7]:
train_input['train'][0]['audio']

{'path': 'zip://│δ└╬│▓┐⌐_│δ└╬┤δ╚¡05_F_nrlee10857_75_├µ├╗_╜╟│╗/│δ└╬│▓┐⌐_│δ└╬┤δ╚¡05_F_nrlee10857_75_├µ├╗_╜╟│╗_04365.wav::/data/freetalk_senior/1.Training/raw_data/2.음성수집도구/2.음성수집도구_1_자유대화(노인남여)_TRAINING.zip',
 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00042725,
        -0.00061035, -0.00073242]),
 'sampling_rate': 16000}

In [8]:
# Too slow
import datasets 

train_label_1 = load_dataset("json", data_files=data_files["train_label"][0])
train_label_2 = load_dataset("json", data_files=data_files["train_label"][1])
train_label_3 = load_dataset("json", data_files=data_files["train_label"][2])

Resolving data files:   0%|          | 0/595637 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/133387 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/80318 [00:00<?, ?it/s]

In [9]:
train_label = datasets.DatasetDict({
    "train": datasets.concatenate_datasets([train_label_1["train"], 
                                            train_label_2["train"], 
                                            train_label_3["train"]])
})

In [10]:
print(train_input, train_label)

DatasetDict({
    train: Dataset({
        features: ['audio'],
        num_rows: 809342
    })
}) DatasetDict({
    train: Dataset({
        features: ['발화정보', '대화정보', '녹음자정보'],
        num_rows: 809342
    })
})


In [11]:
train_dataset = datasets.DatasetDict({
    "train": datasets.concatenate_datasets([train_input['train'], train_label['train']], axis=1)
})

In [32]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', '발화정보', '대화정보', '녹음자정보'],
        num_rows: 809342
    })
})

In [34]:
import pandas as pd

pd_dataset = train_dataset['train'].to_pandas()

# Quick Training


In [14]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1"  # Set the GPU 1 to use
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [15]:
import torch
torch.cuda.device_count()

1

In [23]:
# checkpoint = "jiwon65/whisper-small_korean-zeroth"
checkpoint = "kaen2891/whisper-korean-tiny"

import torch
from transformers import WhisperTokenizer, WhisperFeatureExtractor, WhisperProcessor

# whisper tokenizer is slow
tokenizer = WhisperTokenizer.from_pretrained(checkpoint, language="Korean", task="transcribe")
feature_extractor = WhisperFeatureExtractor.from_pretrained(checkpoint)
processor = WhisperProcessor.from_pretrained(checkpoint, language="Korean", task="transcribe")

def preprocess_dataset(batch):
    audio=batch["audio"]
    label=batch["발화정보"]
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=16000).input_features[0]

    batch["labels"] = tokenizer(label['stt']).input_ids
    return batch

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
try:
    mapped_dataset = train_dataset['train'].map(preprocess_dataset, 
                                               remove_columns=train_dataset.column_names["train"],
                                             # preprocess 함수 내에서 for 문이 돌아서 size가 의미 없다..
                                               # num_proc=16 # os.cpu_count()=44
                                              )
except Exception as e:
    print(f"Error during map: {e}")
print(mapped_dataset)

Map:   0%|          | 0/809342 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
mapped_dataset = mapped_dataset.shuffle(seed=42)
splited_train_dataset = mapped_dataset["train"].train_test_split(test_size=0.2, seed=42)

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        
        batch["labels"] = labels
        
        return batch
        
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

import evaluate

metric = evaluate.load('cer')

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -11] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    cer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}


from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(checkpoint)
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./test_trainer",  # 원하는 리포지토리 이름을 임력한다.
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,  
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=2000,  # epoch 대신 설정
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=200,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer",  # 한국어의 경우 'wer'보다는 'cer'이 더 적합할 것
    greater_is_better=False,
    push_to_hub=False,
)

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=splited_train_dataset["train"],
    eval_dataset=splited_train_dataset["test"],
    # test_dataset=splited_train_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [76]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda:0
Current cuda device: 0
Count of using GPUs: 1


In [ ]:
torch.cuda.empty_cache()

trainer.train()

In [ ]:
trainer.predict()